In [1]:
# 1) Using the Web scraping, read the list of the Sovereign States from 
# https://en.wikipedia.org/wiki/List_of_sovereign_states

In [ ]:
# Creating an empty dataframe with the desired columns

In [2]:
import pandas as pd

In [3]:
columns = ['Country', 'Link', 'Description','Status', 'Capital','Total area' ,'Water area %','Population Density','Population - 2022 estimate','Official languages', 'Ethnic groups', 'Religion', 'Demonym', 'Government','GDP (PPP) - total','GDP (PPP) - per capita', 'GDP (nominal) - total','GDP (nominal) - per capita' ,'HDI','Gini' ,'Currency', 'Time zone','Date format', 'Driving side', 'Calling code', 'ISO code', 'Internet TLD']
df = pd.DataFrame(columns=columns)

In [4]:
import requests
from bs4 import BeautifulSoup
import re

In [5]:
# Making a GET request to the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_sovereign_states'
response = requests.get(url)

In [6]:
# Parsing the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [7]:
# Finding the table containing the list of sovereign states
table = soup.find(class_ = 'sortable wikitable')

In [8]:
# Extracting the names of the countries and their corresponding href links
countries = {}
for row in table.find_all('tr')[3:]:
    cells = row.find_all('td')

    names = cells[0].text
    name = re.split(r"\s*[→–]\s*", names)[0]
    name = name.split(",")[0]
    name = name.split("[")[0]
    name = name.replace("ZZZ","").replace("\xa0","").replace("\n","")
    
    link = cells[0].find('a')
    if link is not None:
            link = link['href']
            countries[name] = link


countries = {k: v for k, v in countries.items() if not v.startswith('#')}
del countries['↑ UN member states and General Assembly observer states ↑']
print(countries)

{'Abkhazia': '/wiki/Abkhazia', 'Afghanistan': '/wiki/Afghanistan', 'Albania': '/wiki/Albania', 'Algeria': '/wiki/Algeria', 'Andorra': '/wiki/Andorra', 'Angola': '/wiki/Angola', 'Antigua and Barbuda': '/wiki/Antigua_and_Barbuda', 'Argentina': '/wiki/Argentina', 'Armenia': '/wiki/Armenia', 'Artsakh': '/wiki/Republic_of_Artsakh', 'Australia': '/wiki/Australia', 'Austria': '/wiki/Austria', 'Azerbaijan': '/wiki/Azerbaijan', 'Bahamas': '/wiki/The_Bahamas', 'Bahrain': '/wiki/Bahrain', 'Bangladesh': '/wiki/Bangladesh', 'Barbados': '/wiki/Barbados', 'Belarus': '/wiki/Belarus', 'Belgium': '/wiki/Belgium', 'Belize': '/wiki/Belize', 'Benin': '/wiki/Benin', 'Bhutan': '/wiki/Bhutan', 'Bolivia': '/wiki/Bolivia', 'Bosnia and Herzegovina': '/wiki/Bosnia_and_Herzegovina', 'Botswana': '/wiki/Botswana', 'Brazil': '/wiki/Brazil', 'Brunei': '/wiki/Brunei', 'Bulgaria': '/wiki/Bulgaria', 'Burkina Faso': '/wiki/Burkina_Faso', 'Burundi': '/wiki/Burundi', 'Cambodia': '/wiki/Cambodia', 'Cameroon': '/wiki/Cameroon

In [13]:
# Loop over the dictionary keys and values
for key, value in countries.items():
#import itertools

#for key, value in itertools.islice(countries.items(), 2):
    # Make a GET request to the website

    country_request = requests.get(('https://en.wikipedia.org' + value))

    website_soup = BeautifulSoup(country_request.content, 'html.parser')
    
    # Find the "mw-parser-output" class
    mw_content_container = website_soup.find('div', class_='mw-content-container')
    first_h2 = mw_content_container.find('h2')
    paragraphs = []
    sibling = first_h2.previous_sibling

    while sibling is not None and sibling.name != 'h2':
        if sibling.name == 'p':
            paragraphs.append(sibling)
        sibling = sibling.previous_sibling
        
    description = ''
    # Extract the text content of each paragraph
    for paragraph in reversed(paragraphs):
        text = paragraph.text.strip()
        description += text
        
# Initialize variables for all the columns you want to extract
    status = ''
    capital = ''
    languages = ''
    ethnic_groups = ''
    religion = ''
    demonym = ''
    government = ''
    hdi = ''
    currency = ''
    time_zone = ''
    driving_side = ''
    calling_code = ''
    iso_code = ''
    internet_tld = ''
    total_area = ''
    water = ''
    population_density =''
    population = ''
    GDPppp = ''
    GDPppp_pc = ''
    GDPnominal = ''
    GDPnominal_pc = ''
    date_format = ''
    gini = ''
    
 # Find the "mw-parser-output" class
    infobox_table = website_soup.find('table', class_='infobox ib-country vcard')

# Find all table rows (tr) in the infobox table
    rows = infobox_table.find_all('tr')
    
# Loop over each row in the info table
    for row in infobox_table.find_all('tr'):
        header = row.find('th', class_='infobox-label')
        data = row.find('td', class_='infobox-data')
        
# Check the header and extract data for the corresponding column
        if header and 'Status' in header.text:
            status = data.text.strip()
        elif header and 'Date format' in header.text:
            date_format = data.text.strip()
        elif header and 'Gini' in header.text:
            gini = data.text.strip() 
        elif header and 'Water' in header.text:
            water = data.text.strip()
            previous_row = row.find_previous_sibling('tr')
            previous_row_cells = previous_row.find_all(['th', 'td'])
            total_area = previous_row_cells[1].text.strip()
        elif header and 'Density' in header.text:
            population_density = data.text.strip()
            previous_row = row.find_previous_sibling('tr')
            previous_row_cells = previous_row.find_all(['th', 'td'])
            population = previous_row_cells[1].text.strip()   
        elif header and 'GDP' in header.text and 'PPP' in header.text:
            next_row = row.find_next_sibling('tr')
            next_row_cells = next_row.find_all(['th', 'td'])
            GDPppp = next_row_cells[1].text.strip()
            nnext_row = next_row.find_next_sibling('tr')
            nnext_row_cells = nnext_row.find_all(['th', 'td'])
            GDPppp_pc = nnext_row_cells[1].text.strip()
        elif header and 'GDP' in header.text and 'nominal' in header.text:
            next_row = row.find_next_sibling('tr')
            next_row_cells = next_row.find_all(['th', 'td'])
            GDPnominal = next_row_cells[1].text.strip()
            nnext_row = next_row.find_next_sibling('tr')
            nnext_row_cells = nnext_row.find_all(['th', 'td'])
            GDPnominal_pc = nnext_row_cells[1].text.strip()
        elif header and 'Capital' in header.text:
            capital = data.text.strip()
        elif header and 'language' in header.text:
            languages = data.text.strip()
        elif header and 'Ethnic' in header.text:
            ethnic_groups = data.text.strip()
        elif header and 'Religion' in header.text:
            religion = data.text.strip()
        elif header and 'Demonym' in header.text:
            demonym = data.text.strip()
        elif header and 'Government' in header.text:
            government = data.text.strip()
        elif header and 'HDI' in header.text:
            hdi = data.text.strip()
        elif header and 'Currency' in header.text:
            currency = data.text.strip()
        elif header and 'Time' in header.text:
            time_zone = data.text.strip()
        elif header and 'Driving' in header.text:
            driving_side = data.text.strip()
        elif header and 'Calling' in header.text:
            calling_code = data.text.strip()
        elif header and 'ISO' in header.text:
            iso_code = data.text.strip()
        elif header and 'Internet' in header.text:
            internet_tld = data.text.strip()

# Append the extracted information to the dataframe
    df = df.append({
            'Country': key,
            'Link': 'https://en.wikipedia.org' + value,
            'Status': status,
            'Capital': capital,
            'Official languages': languages,
            'Ethnic groups': ethnic_groups,
            'Religion': religion,
            'Demonym': demonym,
            'Government': government,
            'HDI': hdi,
            'Currency': currency,
            'Time zone': time_zone,
            'Driving side': driving_side,
            'Calling code': calling_code,
            'ISO 3166 code': iso_code,
            'Internet TLD': internet_tld,
            'Total area' : total_area,
            'Water area %' : water,
            'Population Density' : population_density,
            'Population - 2022 estimate' : population,
            'GDP (PPP) - total' : GDPppp,
            'GDP (PPP) - per capita' : GDPppp_pc,
            'GDP (nominal) - total' : GDPnominal,
            'GDP (nominal) - per capita' : GDPnominal_pc,
            'Date format' : date_format, 
            'Gini' : gini,
            'Country': key,
            'Link': 'https://en.wikipedia.org' + value,
            'Description': description

        }, ignore_index=True)


AttributeError: 'DataFrame' object has no attribute 'append'

In [15]:
df

,Country,Link,Description,Status,Capital,Total area,Water area %,Population Density,Population - 2022 estimate,Official languages,...,GDP (nominal) - per capita,HDI,Gini,Currency,Time zone,Date format,Driving side,Calling code,ISO code,Internet TLD


In [16]:
df.to_csv('countries_data.csv', index=False)

In [17]:
df.to_excel('countries.xlsx', index=False)